<a href="https://colab.research.google.com/github/salim-hbk/ai-ml/blob/main/Fmnist_customTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from tqdm import tqdm
import tensorflow_datasets as tfds
import matplotlib.ticker as mticker
from tensorflow.keras.activations import relu, softmax

In [87]:
#Get the data and process the data
train_data = tfds.load('fashion_mnist' , split='train')
test_data  = tfds.load('fashion_mnist' , split="test")

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteZS1S5I/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteZS1S5I/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [88]:
def process_data(data):
    image = data['image']
    image = tf.reshape(image , [-1])
    image = tf.cast(image , tf.float32)
    image = image / 255.0

    return image , data['label']

In [89]:
train_data = train_data.map(process_data)
test_data  = test_data.map(process_data)

In [90]:
batch_size = 64

train_data = train_data.shuffle(buffer_size=1024).batch(batch_size)
test_data  = test_data.batch(batch_size)

In [91]:
class_names = ["T-shirt/top", "Trouser/pants", "Pullover shirt", "Dress", "Coat", 
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [92]:
def base_model():
    input  = Input(shape=(28*28 , ) , name = "input_layer")
    x = Dense(64 , activation=relu , name = "dense1")(input)
    x = Dense(64 , activation=relu , name = "dense2")(x)
    output = Dense(10 , activation=softmax , name="output_layer")(x)
    model = Model(inputs = input , outputs = output)
    return model

In [96]:

loss_obj       = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer_obj  = tf.keras.optimizers.Adam()

train_acc_matrix = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_matrix   = tf.keras.metrics.SparseCategoricalAccuracy()

In [97]:
def run_optimizer(model , x , y_true):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss_val = loss_obj(y_true=y_true , y_pred=logits)
    grad = tape.gradient(loss_val , model.trainable_weights)
    optimizer_obj.apply_gradients(zip(grad , model.trainable_weights))

    return logits , loss_val

In [98]:
def train_one_epoch(model , train_data):
    losses = []
    pbar = tqdm(total=len(list(enumerate(train_data))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
    for batch_no , (data , label) in enumerate(train_data):
        y_pred , loss = run_optimizer(model , data , label)
        losses.append(loss)
        train_acc_matrix(label , y_pred)
        pbar.set_description("Training loss for step %s: %.4f" % (int(batch_no), float(loss)))
        pbar.update()
    return losses

In [99]:
def perform_validation(model , test_data):
    losses = []
    for (data , label) in test_data:
        y_pred = model(data)
        loss = loss_obj(label , y_pred)
        losses.append(loss)
        val_acc_matrix(label , y_pred)
    return losses

In [100]:
def train(train , test , epochs = 10):
    model = base_model()

    history = {}
    history['train_loss'] = []
    history['val_loss'] = []

    history['train_acc'] = []
    history['val_acc'] = []

    val_epoch_loss   = []
    for epoch in range(epochs):
        print('Start of epoch %d' % (epoch,))

        train_losses = train_one_epoch(model , train_data=train)
        train_acc    = train_acc_matrix.result()
        history['train_acc'].append(train_acc.numpy())
        train_acc_matrix.reset_states()

        val_losses   = perform_validation(model , test_data=test)
        val_acc      = val_acc_matrix.result()
        history['val_acc'].append(val_acc.numpy())
        val_acc_matrix.reset_states()

        history['train_loss'].append(np.mean(train_losses))
        history['val_loss'].append(np.mean(val_losses))

        print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f,\
         Train Accuracy: %.4f, Validation Accuracy %.4f' % (epoch, float(np.mean(train_losses)), float(np.mean(val_losses)),
                                                            float(train_acc), float(val_acc)))

    history['model'] = model
    return history

In [103]:
history = train(train_data , test_data)

Start of epoch 0


Training loss for step 937: 0.2319: 100%|█████████▉| 937/938 


 Epoch 0: Train loss: 0.5475  Validation Loss: 0.4460,         Train Accuracy: 0.8095, Validation Accuracy 0.8439
Start of epoch 1


Training loss for step 937: 0.1135: 100%|█████████▉| 937/938 


 Epoch 1: Train loss: 0.3953  Validation Loss: 0.3984,         Train Accuracy: 0.8581, Validation Accuracy 0.8595
Start of epoch 2


...


 Epoch 7: Train loss: 0.2818  Validation Loss: 0.3529,         Train Accuracy: 0.8961, Validation Accuracy 0.8720
Start of epoch 8


Training loss for step 937: 0.1356: 100%|█████████▉| 937/938 


 Epoch 8: Train loss: 0.2724  Validation Loss: 0.3525,         Train Accuracy: 0.8989, Validation Accuracy 0.8761
Start of epoch 9


Training loss for step 937: 0.1244: 100%|█████████▉| 937/938 


 Epoch 9: Train loss: 0.2610  Validation Loss: 0.3439,         Train Accuracy: 0.9024, Validation Accuracy 0.8799
